<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/17_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install eciespy eth_keys cryptos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.9 MB/s 
     |████████████████████████████████| 555 kB 15.3 MB/s 
     |████████████████████████████████| 2.0 MB 31.3 MB/s 
     |████████████████████████████████| 105 kB 62.7 MB/s 
     |████████████████████████████████| 481 kB 59.8 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1236692 sha256=d801ae332e544395534917224a8e36aa1cffb8e759311543fe273e0c9a38e666
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca67522048d695b556bba4f59
Successfully built cytoolz


In [2]:
import ecies
import eth_keys
import coincurve
import hashlib

In [19]:
broad_message=b'''Tell the world something. Encrypt with AES key.
AES encrypt multiple copes of that key using a list of shared secrets.'''
print(broad_message.decode(),len(broad_message))

Tell the world something. Encrypt with AES key.
AES encrypt multiple copes of that key using a list of shared secrets. 118


In [39]:
from coincurve.utils import get_valid_secret

master_aes=get_valid_secret()
broad_message_aes=ecies.aes_encrypt(master_aes,broad_message)

Now we have an encrypted message and we want to distribute it so select key holders can open it. 

We will encrypt the `master_aes` encryption key (using aes) so that each recipient can access it using their special-unique encrypted copy.

**HOT DIGGITY DOG !!! HOW DO WE MAKE SURE ONLY THE SELECT KEY HOLDER CAN OPEN THEIR COPY ???**

#Diffie Helman Key Exchange...Broadcast

In [20]:
privKey1 = ecies.utils.generate_eth_key()
privKey2 = ecies.utils.generate_eth_key()
privKey3 = ecies.utils.generate_eth_key()
print(privKey1, type(privKey1))
print(privKey2, type(privKey2))
print(privKey3, type(privKey3))

0x56b61df9c372f3a4c59fc986c7c1745c6b733de1dd8cdb8ce0dfaab39a48b86c <class 'eth_keys.datatypes.PrivateKey'>
0x6655df8927cd5df81737292294210336eb80298d379d5a17b105f60df0eb5751 <class 'eth_keys.datatypes.PrivateKey'>
0xa4d3efc650a78d101a9581c24dd273c95511783c58d21b8cc42bdbbbac060623 <class 'eth_keys.datatypes.PrivateKey'>


In [41]:
#convert from ecies private/public key to concurve private/public key

k1 = coincurve.PrivateKey(privKey1.to_bytes())
k2 = coincurve.PrivateKey(privKey2.to_bytes())
k3 = coincurve.PrivateKey(privKey2.to_bytes())

p1 = coincurve.PublicKey( privKey1.public_key.to_compressed_bytes() )
p2 = coincurve.PublicKey( privKey2.public_key.to_compressed_bytes() )
p3 = coincurve.PublicKey( privKey3.public_key.to_compressed_bytes() )

In [42]:
shared12 = p2.multiply(k1.secret)
shared21 = p1.multiply(k2.secret)

In [23]:
shared12.format().hex(),shared21.format().hex()

('02097d7834368fbdc1c1e016a5301a9f71be7ed7578fa0aa63ff43e1755b0f34b8',
 '02097d7834368fbdc1c1e016a5301a9f71be7ed7578fa0aa63ff43e1755b0f34b8')

In [32]:
from Crypto.Protocol.KDF import HKDF
AES_KEY_BYTES_LEN = 32
from Crypto.Hash import SHA256

In [44]:
ps=[p1,p2,p3]
shared_secrets=[p.multiply(k1.secret) for p in ps]

In [67]:
def shared_key(prvKey,pubKey):
  cc_prvKey=coincurve.PrivateKey(prvKey.to_bytes())
  cc_pubKey= coincurve.PublicKey( pubKey.to_compressed_bytes() )
  return HKDF(cc_pubKey.multiply(cc_prvKey.secret).format(), AES_KEY_BYTES_LEN, b"", SHA256)

In [73]:
shared_key(privKey3,privKey1.public_key)

b'\xfcc/\n\x89\x1a\x16\xd8\xdbL\x9a\x00\xb2\x0b:\xfe\xdc\xd3"\x85\x0b\x90\xca_\xe7\x91\x1c\xdf0[J\xc4'

In [ ]:
shared_keys=[ HKDF(shared_secret.format(), AES_KEY_BYTES_LEN, b"", SHA256) for shared_secret in shared_secrets]

In [47]:
access_keys=[ ecies.aes_encrypt(shared_key,master_aes) for shared_key in shared_keys]
print(len(b''.join(access_keys)))
access_keys

192


[b'`\xcc"\xcc\xd6]\x98M^\xc5\x88\x01\x0b\xd49G\xedG\tK\xa8\xfc(\x1dC\xces1,wr\xd8!\xe1.\xae\x1c\xae#7\xd6\xb5(\xa4R\x9f\xfe\xce\x89-d\x7f\xb0\xfb\xae?s\xe4X\xd4\xae\xc3\xc8\x85',
 b'\x8a\xe1\xe4\x91\x1e"kP"\x82e\xb9\xde\xcd\xc8A\xcaq\x8a`\xc2\xa5\x87\x00\xb65\xcb\xce\x8b\xbc\x1e\x15\xc2\xdei\x81\x89qn\xbb0.\xfd\xff\x88\x86A\x86\xc3L,\xf6\x83\xfe\xca\x13A\xf7\x08\x13\xd2oc\xbb',
 b'\x9bTZ\x08\x8f\xa5\xc1\xccY\x0c.h\xd7\xe5\x85\x8f\x92\x13\xa4\x9c\xae\x8a\x17\x1dY\xb6\xa5\xcf\x17\x04\x15\xd0\x0f\n_\xc5\xb5b2jt&5\x8b\x12\x98\xf2\x08NE\x8c7\xab\x9cTT\xd9^\xac\xa5\x94\xad\xc7*']

In [59]:
ecies.aes_decrypt(shared_keys[0],access_keys[0])

b'Nv\x07a_\xb3\xabwA\x97\x85A\x04i\n\x00\xfc\xdf5\xb3c+\x03\\}\xbbA\x8a8\xbb\xbf\xe3'

In [61]:
dist_keys=[ecies.aes_decrypt(*args)  for args in zip(shared_keys,access_keys)]
dist_keys

[b'Nv\x07a_\xb3\xabwA\x97\x85A\x04i\n\x00\xfc\xdf5\xb3c+\x03\\}\xbbA\x8a8\xbb\xbf\xe3',
 b'Nv\x07a_\xb3\xabwA\x97\x85A\x04i\n\x00\xfc\xdf5\xb3c+\x03\\}\xbbA\x8a8\xbb\xbf\xe3',
 b'Nv\x07a_\xb3\xabwA\x97\x85A\x04i\n\x00\xfc\xdf5\xb3c+\x03\\}\xbbA\x8a8\xbb\xbf\xe3']

In [62]:
master_aes

b'Nv\x07a_\xb3\xabwA\x97\x85A\x04i\n\x00\xfc\xdf5\xb3c+\x03\\}\xbbA\x8a8\xbb\xbf\xe3'

In [64]:
ecies.aes_decrypt(dist_keys[2],broad_message_aes)

b'Tell the world something. Encrypt with AES key.\nAES encrypt multiple copes of that key using a list of shared secrets.'

In [75]:
mess2post=len(access_keys).to_bytes(1,'big')+b''.join(access_keys)+broad_message_aes

In [76]:
mess2post

b'\x03`\xcc"\xcc\xd6]\x98M^\xc5\x88\x01\x0b\xd49G\xedG\tK\xa8\xfc(\x1dC\xces1,wr\xd8!\xe1.\xae\x1c\xae#7\xd6\xb5(\xa4R\x9f\xfe\xce\x89-d\x7f\xb0\xfb\xae?s\xe4X\xd4\xae\xc3\xc8\x85\x8a\xe1\xe4\x91\x1e"kP"\x82e\xb9\xde\xcd\xc8A\xcaq\x8a`\xc2\xa5\x87\x00\xb65\xcb\xce\x8b\xbc\x1e\x15\xc2\xdei\x81\x89qn\xbb0.\xfd\xff\x88\x86A\x86\xc3L,\xf6\x83\xfe\xca\x13A\xf7\x08\x13\xd2oc\xbb\x9bTZ\x08\x8f\xa5\xc1\xccY\x0c.h\xd7\xe5\x85\x8f\x92\x13\xa4\x9c\xae\x8a\x17\x1dY\xb6\xa5\xcf\x17\x04\x15\xd0\x0f\n_\xc5\xb5b2jt&5\x8b\x12\x98\xf2\x08NE\x8c7\xab\x9cTT\xd9^\xac\xa5\x94\xad\xc7*\xc5\xae\xe0=D1\xef&m\xcf\xd9\\\x0b\xc2\xde\xa2\x90l\xd1\x18x\xdb\xb4\x80\xf3\xc0m\x91^1\xa8\xdf5!\x1d)\xe1\xc7\xfeV\x93\xc9\xfb\xc1E\xce\xa4A\xf8\x86\xca<38\xbe\x1f\xf1\xe6%\xd0<\xdf^\xb819\x9b,\xbc\xc6p@\xac\'\xcd\xa1\x01i\x14\xb0Q\x91\x9ah\x99\xa4\xbb\xf5\r\x9b\x86\xbf\xa4n\xec\xe0\xa4&\xc8\xa7b\xd1\x8f\xb821\xe4@k\xf8]5\x0b\xc5\xcd\x82\x06l\xd1\xac\t2Y\x040\x15\xf5\xa19\x8dEUq\xaa~\xf2\xdbc\xfd5\x80vu\x81\xb8\x0cb\xddZ\xa8'

In [80]:
len(mess2post)

343

In [81]:
len(broad_message)

118

For encrypted transmissions to single recipients...it is possible to encrypt directly with the shared key (derived from shared secret). This way both sender and recipient will have access without needing to post anything else.

Sometimes we may post an AES encrypted message without any intended recipient. In this case we will use a temporary AES key and encrypt for ourselves using the same protocol. It could be for personal access or the session AES key could be published at some point in the future. In other words, the encoding protocol is identical for the case when the only recipient is ourselves.